In [9]:
!pip install gdown pandas
import os
os.makedirs("data", exist_ok=True)

In [10]:
!gdown --id 1ExuBzkObUNmqgmeaVMF-OqnAuo9yuozK -O data/All_capped_keywords.zip

/usr/local/lib/python3.12/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ExuBzkObUNmqgmeaVMF-OqnAuo9yuozK
From (redirected): https://drive.google.com/uc?id=1ExuBzkObUNmqgmeaVMF-OqnAuo9yuozK&confirm=t&uuid=e5931464-4d76-404c-a18a-ef3c329ff993
To: /content/data/All_capped_keywords.zip
100% 579M/579M [00:07<00:00, 77.4MB/s]


In [11]:
import zipfile

with zipfile.ZipFile("data/All_capped_keywords.zip", "r") as z:
    z.extractall("data/cspapersum")
    print("Extracted files:", len(z.namelist()))
    print("Sample files:", z.namelist()[:20])


Extracted files: 1
Sample files: ['All_capped_keywords.csv']


In [12]:
import glob, pandas as pd

csvs = glob.glob("data/cspapersum/**/*.csv", recursive=True)
print("Found CSVs:", csvs[:5])

for path in csvs:
    try:
        df = pd.read_csv(path, nrows=3)
        print(path, "→", df.columns.tolist())
    except Exception as e:
        print("Could not load", path, e)


Found CSVs: ['data/cspapersum/All_capped_keywords.csv']
data/cspapersum/All_capped_keywords.csv → ['paperID', 'venue', 'year', 'openAccessPdf', 'url', 'authors', 'referenceCount', 'title', 'abstract', 'conclusion', 'Chatgpt Response', 'Key Takeaways', 'Importance', 'Model/Method Proposed', 'Performance', 'Effectiveness', 'Future Works', 'Sentiment', 'Sentiment Score', 'combined', 'combined_keywords', 'response_keywords', 'future_work_keywords', 'capped_keywords', 'field']


In [13]:
import pandas as pd

df = pd.read_csv("data/cspapersum/All_capped_keywords.csv")
df = df[['title', 'abstract']]

df.to_csv("data/titles_abstracts.csv", index=False)

print("Saved clean dataset.")
print("Number of rows: ", len(df))
df.head()

Saved clean dataset.
Number of rows:  91919


,title,abstract
0,Generalizable Representation Learning for Mixt...,Face anti-spoofing approach based on domain ge...
1,Coordinating Human and Agent Behavior in Colle...,\n \n Various social situations entail a colle...
2,Adverse Drug Reaction Prediction with Symbolic...,\n \n Adverse drug reaction (ADR) is a major b...
3,TempLe: Learning Template of Transitions for S...,Transferring knowledge among various environme...
4,Scalable Recollections for Continual Lifelong ...,Given the recent success of Deep Learning appl...


In [13]:
import pandas as pd, os
df = pd.read_csv("data/titles_abstracts.csv")
print("Full size (rows):", len(df))
sample = df.head(1000)                     # adjust if you want
sample_path = "data/titles_abstracts_sample.csv"
sample.to_csv(sample_path, index=False)
print("Wrote:", sample_path)


Full size (rows): 91919
Wrote: data/titles_abstracts_sample.csv


In [18]:
!pip -q install pandas numpy transformers
import re, pandas as pd, numpy as np
df = pd.read_csv("data/titles_abstracts.csv")
print(len(df), "rows"); df.head(3)

91919 rows


,title,abstract
0,Generalizable Representation Learning for Mixt...,Face anti-spoofing approach based on domain ge...
1,Coordinating Human and Agent Behavior in Colle...,\n \n Various social situations entail a colle...
2,Adverse Drug Reaction Prediction with Symbolic...,\n \n Adverse drug reaction (ADR) is a major b...


In [19]:
#Heuristic

PLAYFUL_WORDS = {
    #humor/slang
    "haha", "lol", "lmao", "rofl", "silly", "goofy", "whimsical", "jest", "chuckle", "giggle", "smirk", "banter", "slapstick", "sarcasm", "wit", "irony", "satire", "parody", "observational", "cringe", "deadpan", "absurd", "exaggeration", "understatement",
    #vibe
    "ridiculously", "insanely", "super", "hella", "wicked", "terribly", "unbelievably", "totally", "absolutely", "so", "dead", "mad", "lowkey", "legit", "literally", "actually", "for real", "for sure", "dude", "bro", "sick", "fire", "epic", "wild",
    #metaphors / tropes
    "hack","trick","cookbook","recipe","alchemy","wizard","beast","frankenstein",
    "zoo","odyssey","tales","story","magic","myth","saga","adventures",
    "to rule them all", "the good, the bad"
}

PATTERNS = [
    re.compile(r".+:\s+(?:a|the)\s+(?:tale|story|odyssey|cookbook)", re.I),
    re.compile(r"the good, the bad(?:, and the ugly)?", re.I),
    re.compile(r"to rule (?:them|it) all", re.I),
    re.compile(r"[;:?!]{2,}"),                  # multiple ? or !
    re.compile(r"[A-Za-z]+\s*&\s*[A-Za-z]+"),   # “X & Y”
    re.compile(r"\([^)]{0,30}\)"),              # brief parenthetical
    re.compile(r":\s*\)", re.I),                # smiley-ish ": )"
]
FORMAL_STARTS = tuple(["on the","towards","an analysis of","a study of","investigating","notes on"])

def rule_playful_score(title: str) -> float:
    t = str(title).strip()
    tl = t.lower()
    score = 0.0
    for w in PLAYFUL_WORDS:
        if w in tl: score += 1.0
    for p in PATTERNS:
        if p.search(t): score += 0.8
    if "!" in t: score += 0.5
    if "?" in t: score += 0.3
    if tl.startswith(FORMAL_STARTS): score -= 0.8
    return round(score, 3)

df["rule_score"] = df["title"].astype(str).apply(rule_playful_score)

# quick output check
df.sort_values("rule_score", ascending=False).head(8)[["title","rule_score"]]

,title,rule_score
57508,Into the Wild with AudioScope: Unsupervised Au...,4.0
89480,Facebook (A)Live?: Are Live Social Broadcasts ...,3.9
15854,"When did you become so smart, oh wise one?! Sa...",3.6
50551,What Could Go Wrong?! 2nd Workshop: Lessons Le...,3.6
51796,Anthropomorphize me!: Effects of Robot Gender ...,3.3
51128,Cheating with a Socially Assistive Robot?: A M...,3.1
10916,Is a Knowledge-based Response Engaging?: An An...,3.1
61390,How Magic Is Zero?: An Empirical Analysis of I...,3.1


In [3]:
!pip -q install transformers
from transformers import pipeline
import torch, numpy as np

# Config
HAS_GPU    = torch.cuda.is_available()
MODEL_NAME = "facebook/bart-large-mnli" if HAS_GPU else "typeform/distilbert-base-uncased-mnli"
DEVICE     = 0 if HAS_GPU else -1
BATCH_SIZE = 48 if HAS_GPU else 12

LABELS  = ["playful", "neutral"]
HYP     = "This paper title is {}."
ZS_THRESH = 0.60
USE_CASCADE = True
LOW, HIGH   = 0.8, 2.5   # "maybe" band for rule_score

zs = pipeline("zero-shot-classification", model=MODEL_NAME, device=DEVICE)

# Select which titles to score
if USE_CASCADE and "rule_score" in df.columns:
    mask = df["rule_score"].between(LOW, HIGH)
    idx = df.index[mask]
else:
    idx = df.index

titles = df.loc[idx, "title"].fillna("").astype(str).tolist()
if len(titles) == 0:
    # Nothing to score (maybe band empty)
    if "zs_playful_prob" not in df.columns:
        df["zs_playful_prob"] = 0.0
else:
    outs = zs(
        titles,
        candidate_labels=LABELS,
        hypothesis_template=HYP,
        multi_label=False,
        truncation=True,
        batch_size=BATCH_SIZE  # <-- key for speed
    )
    p_idx = outs[0]["labels"].index("playful")
    probs = np.array([float(o["scores"][p_idx]) for o in outs], dtype=float)

    if len(idx) != len(df):
        # cascade: fill only scored rows; others = 0.0
        if "zs_playful_prob" not in df.columns:
            df["zs_playful_prob"] = 0.0
        df.loc[idx, "zs_playful_prob"] = probs
    else:
        df["zs_playful_prob"] = probs

df["playful_zs_flag"] = df["zs_playful_prob"] >= ZS_THRESH
df.sort_values("zs_playful_prob", ascending=False).head(10)[["title","zs_playful_prob"]]


,title,zs_playful_prob
22304,"Better, Funner, Stronger: A Gameful Approach t...",0.998467
22161,The Critical Catalog: Library Information Syst...,0.996914
20437,Irony of The Phoenix Project (2008-2015) in th...,0.996387
15992,Trick Me If You Can: Adversarial Writing of Tr...,0.996359
17313,Hogwild! over Distributed Local Data Sets with...,0.995090
51357,Let's Make this Fun!: Activities to Motivate C...,0.995064
31414,JAWS: Just A Wild Shot for Cinematic Transfer ...,0.994956
88526,Lie to Me: Abusing the Mobile Content Sharing ...,0.994933
55321,FunnyBirds: A Synthetic Vision Dataset for a P...,0.994814
50469,A Prototype of a Robot Memory Game: Exploring ...,0.994682


In [4]:
import math, os
if "rule_score" not in df.columns:
    df["rule_score"] = 0.0

def squash(x, center=1.5):
    return 1 / (1 + math.exp(-(x - center)))  # map rule_score ~ to [0,1]

df["rule_norm"] = df["rule_score"].apply(squash)

df["ensemble_score"] = 0.5*df["rule_norm"] + 0.5*df["zs_playful_prob"]

ENSEMBLE_THRESH = 0.55   # ↑ for precision, ↓ for recall
df["playful_flag"] = df["ensemble_score"] >= ENSEMBLE_THRESH

# quick stats
print(f"Flagged (ensemble): {df['playful_flag'].mean():.2%} of {len(df)} titles")

# save outputs
os.makedirs("results", exist_ok=True)
df[df["playful_flag"]].to_csv("results/playful_titles.csv", index=False)
df[~df["playful_flag"]].sample(min(30, (~df["playful_flag"]).sum()), random_state=7)\
  .to_csv("results/neutral_title_examples.csv", index=False)

print("Saved: results/playful_titles.csv and results/neutral_title_examples.csv")


Flagged (ensemble): 1.72% of 91919 titles
Saved: results/playful_titles.csv and results/neutral_title_examples.csv


In [8]:
!pip -q install -U "transformers>=4.41,<5" datasets accelerate rouge-score nltk unidecode

import os, re, numpy as np, pandas as pd, torch, nltk
from datasets import Dataset, DatasetDict
from transformers import (
    AutoTokenizer, AutoModelForSeq2SeqLM,
    DataCollatorForSeq2Seq, Seq2SeqTrainer, Seq2SeqTrainingArguments
)
from rouge_score import rouge_scorer
from unidecode import unidecode

nltk.download("punkt")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [9]:
# Load your clean data
df = pd.read_csv("data/titles_abstracts.csv").dropna(subset=["abstract","title"]).copy()

def clean(s):
    s = unidecode(str(s))
    s = re.sub(r"\s+", " ", s).strip()
    return s

df["abstract"] = df["abstract"].apply(clean)
df["title"]    = df["title"].apply(clean)

# (Optional) keep a manageable subset for the first run
MAX_EXAMPLES = min(len(df), 4000)
df = df.sample(MAX_EXAMPLES, random_state=42).reset_index(drop=True)

# Split
split = int(0.9 * len(df))
train_df, valid_df = df.iloc[:split], df.iloc[split:]

ds = DatasetDict({
    "train": Dataset.from_pandas(train_df[["abstract","title"]]),
    "validation": Dataset.from_pandas(valid_df[["abstract","title"]]),
})

# Tokenization
MODEL_NAME = "google/flan-t5-small"  # bump to flan-t5-base if you have more VRAM
tok = AutoTokenizer.from_pretrained(MODEL_NAME)

MAX_IN, MAX_OUT = 512, 32
PROMPT_PREFIX = "Write a concise, specific academic paper title for the abstract.\nAbstract: "

def preprocess(batch):
    inputs  = [PROMPT_PREFIX + a for a in batch["abstract"]]
    targets = batch["title"]
    model_inputs = tok(inputs, max_length=MAX_IN, truncation=True)
    with tok.as_target_tokenizer():
        labels = tok(targets, max_length=MAX_OUT, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized = ds.map(preprocess, batched=True, remove_columns=["abstract","title"])


Map:   0%|          | 0/3600 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/400 [00:00<?, ? examples/s]

In [10]:
model = AutoModelForSeq2SeqLM.from_pretrained(MODEL_NAME)
data_collator = DataCollatorForSeq2Seq(tok, model=model)

args = Seq2SeqTrainingArguments(
    output_dir="checkpoints/flan_t5_sft",
    per_device_train_batch_size=8,       # lower to 4 if OOM
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,
    learning_rate=5e-5,
    num_train_epochs=2,
    warmup_ratio=0.05,
    logging_steps=50,
    save_total_limit=2,
    fp16=torch.cuda.is_available(),      # mixed precision on GPU
    report_to=[],                        # no wandb
    # NOTE: we skip evaluation_strategy/eval_steps to avoid version issues.
    # We'll evaluate manually right after training.
)

trainer = Seq2SeqTrainer(
    model=model,
    args=args,
    train_dataset=tokenized["train"],
    eval_dataset=tokenized["validation"],   # present but no auto-eval during training
    data_collator=data_collator,
    tokenizer=tok,
)

trainer.train()


/tmp/ipython-input-30096522.py:20: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Seq2SeqTrainer.__init__`. Use `processing_class` instead.
  trainer = Seq2SeqTrainer(


Step,Training Loss
50,0.000000
100,0.000000
150,0.000000
200,0.000000
250,0.000000
300,0.000000
350,0.000000
400,0.000000
450,0.000000


TrainOutput(global_step=450, training_loss=0.0, metrics={'train_runtime': 247.804, 'train_samples_per_second': 29.055, 'train_steps_per_second': 1.816, 'total_flos': 980604362244096.0, 'train_loss': 0.0, 'epoch': 2.0})

In [11]:
# Helper: generate one title
def gen_title(abstract, max_new_tokens=32, num_beams=4):
    inp = tok(PROMPT_PREFIX + abstract, return_tensors="pt", truncation=True, max_length=MAX_IN).to(model.device)
    out = model.generate(**inp, max_new_tokens=32, num_beams=num_beams)
    return tok.decode(out[0], skip_special_tokens=True).strip()

# Sample N examples from validation
N = min(200, len(valid_df))
sample = valid_df.head(N).copy()
sample["gen_title"] = sample["abstract"].apply(gen_title)

# ROUGE-L (rough overlap signal)
scorer = rouge_scorer.RougeScorer(["rougeL"], use_stemmer=True)
sample["rougeL"] = sample.apply(lambda r: scorer.score(r["title"], r["gen_title"])["rougeL"].fmeasure, axis=1)

print("Mean ROUGE-L on sample:", round(sample["rougeL"].mean(), 3))
sample.head(5)[["title","gen_title","rougeL"]]


Mean ROUGE-L on sample: 0.277


,title,gen_title,rougeL
3600,Semi-Supervised Learning on Meta Structure: Mu...,Unify multi-view learning with multi-view lear...,0.173913
3601,Learning Energy-Based Model with Variational A...,Asymmetrical auto-encoder for MCMC learning.,0.235294
3602,Positive and Negative Opinions About Living wi...,A survey of the positive and negative opinions...,0.533333
3603,Diverse and Specific Clarification Question Ge...,KPCNet generates more specific questions and p...,0.173913
3604,APrompt: Attention Prompt Tuning for Efficient...,"A Novel Attention Prompt Tuning Method, namely...",0.785714


In [12]:
os.makedirs("results", exist_ok=True)
sample.to_csv("results/sft_titles_validation_sample.csv", index=False)
trainer.save_model("checkpoints/flan_t5_sft/best")
print("Saved:")
print("- results/sft_titles_validation_sample.csv")
print("- checkpoints/flan_t5_sft/best (model + tokenizer)")


Saved:
- results/sft_titles_validation_sample.csv
- checkpoints/flan_t5_sft/best (model + tokenizer)
